In [109]:
import pandas as pd
import numpy as np
from mypipes import *
testd = pd.read_csv("/home/pavan117/Courses/Edvancer 53k course/2. Python/Projects/Project 1/Consumer_Complaints_test_share.csv")
traind = pd.read_csv("/home/pavan117/Courses/Edvancer 53k course/2. Python/Projects/Project 1/Consumer_Complaints_train.csv")

In [110]:
'''Combining the datasets to ensure uniform data transformations
    1. Add target var to test data.
    2. Add a data column so that i can properly separate them later
    3. Concat them
'''
testd['Consumer disputed?'] = np.nan
testd['data'] = 'test'
traind['data'] = 'train'
fulldata = pd.concat([traind, testd], axis=0)

/home/pavan117/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [111]:
'''Checking for missing values in each column''' '''I ran it, dropped it and ran again, thus all useless columns are now gone'''
cols_to_drop = []
for col in fulldata.columns:
    s = sum(pd.isnull(fulldata[col]))
    if s/598027 >0.5:
        cols_to_drop.append(col)
print(cols_to_drop)    

['Company public response', 'Consumer complaint narrative', 'Consumer consent provided?', 'Sub-issue', 'Tags']


In [112]:
'''Those Columns which have more than 50% missing values'''
fulldata = fulldata.drop(cols_to_drop, axis=1)

In [153]:
pd.value_counts(fulldata['data'])

train    478421
test     119606
Name: data, dtype: int64

In [368]:
fulldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598027 entries, 0 to 119605
Data columns (total 41 columns):
Consumer disputed?                                              598027 non-null int64
Timely response?                                                598027 non-null int64
data                                                            598027 non-null object
delay                                                           598027 non-null int64
Company_Bank of America                                         598027 non-null int64
Company_Wells Fargo & Company                                   598027 non-null int64
Company_JPMorgan Chase & Co.                                    598027 non-null int64
Company_Equifax                                                 598027 non-null int64
Company_Experian                                                598027 non-null int64
Company_TransUnion Intermediate Holdings, Inc.                  598027 non-null int64
Company_Citibank            

In [117]:
del fulldata['Complaint ID']
del fulldata['ZIP code']

In [232]:
fulldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598027 entries, 0 to 119605
Data columns (total 11 columns):
Company                         598027 non-null object
Company response to consumer    598027 non-null object
Consumer disputed?              478421 non-null object
Issue                           598027 non-null object
Product                         598027 non-null object
State                           598027 non-null object
Sub-product                     598027 non-null object
Submitted via                   598027 non-null object
Timely response?                598027 non-null int64
data                            598027 non-null object
delay                           598027 non-null int64
dtypes: int64(2), object(9)
memory usage: 54.8+ MB


In [125]:
delays = (pd.to_datetime(fulldata['Date sent to company']) - pd.to_datetime(fulldata['Date received'])).dt.days
delays.head()
fulldata['delay'] = delays
del fulldata['Date sent to company']
del fulldata['Date received']

In [129]:
fulldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598027 entries, 0 to 119605
Data columns (total 11 columns):
Company                         598027 non-null object
Company response to consumer    598027 non-null object
Consumer disputed?              478421 non-null object
Issue                           598027 non-null object
Product                         598027 non-null object
State                           593263 non-null object
Sub-product                     424802 non-null object
Submitted via                   598026 non-null object
Timely response?                598027 non-null object
data                            598027 non-null object
delay                           598027 non-null int64
dtypes: int64(1), object(10)
memory usage: 54.8+ MB


In [131]:
temp = (fulldata['Timely response?']=="Yes").astype(int)
temp.head()
fulldata['Timely response?'] = temp

In [132]:
fulldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598027 entries, 0 to 119605
Data columns (total 11 columns):
Company                         598027 non-null object
Company response to consumer    598027 non-null object
Consumer disputed?              478421 non-null object
Issue                           598027 non-null object
Product                         598027 non-null object
State                           593263 non-null object
Sub-product                     424802 non-null object
Submitted via                   598026 non-null object
Timely response?                598027 non-null int64
data                            598027 non-null object
delay                           598027 non-null int64
dtypes: int64(2), object(9)
memory usage: 54.8+ MB


In [148]:
impute_these = {}
for col in fulldata.columns:
    if col in ['Consumer disputed?','data']:
        continue
    s = sum(pd.isnull(fulldata[col]))
    if s!=0:
        impute_these[col] = 'missing'
print(impute_these)

{'State': 'missing', 'Sub-product': 'missing', 'Submitted via': 'missing'}


In [137]:
backup = fulldata.copy()

In [144]:
pd.value_counts(fulldata['data'])

train    478421
test     119606
Name: data, dtype: int64

In [152]:
fulldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598027 entries, 0 to 119605
Data columns (total 11 columns):
Company                         598027 non-null object
Company response to consumer    598027 non-null object
Consumer disputed?              478421 non-null object
Issue                           598027 non-null object
Product                         598027 non-null object
State                           598027 non-null object
Sub-product                     598027 non-null object
Submitted via                   598027 non-null object
Timely response?                598027 non-null int64
data                            598027 non-null object
delay                           598027 non-null int64
dtypes: int64(2), object(9)
memory usage: 54.8+ MB


In [151]:
fulldata = fulldata.fillna(impute_these)

In [154]:
trainingdata = fulldata.loc[fulldata['data']=="train"]
testdata = fulldata.loc[fulldata['data']=="test"]

In [156]:
cat_vars = []
num_vars = []
for col in fulldata.columns:
    if col in ['Consumer disputed?']:
        continue
    elif fulldata[col].dtype == "O":
        cat_vars.append(col)
    else:
        num_vars.append(col)

NameError: name 'car_vars' is not defined

In [159]:
backup = fulldata.copy()

In [185]:
cat_vars = cat_vars[:-1]

In [187]:
all_other_vars = []
for col in fulldata.columns:
    if col not in cat_vars:
        all_other_vars.append(col)
all_other_vars

['Consumer disputed?', 'Timely response?', 'data', 'delay']

In [211]:
all_other_vars= ['Consumer disputed?', 'Timely response?', 'delay']

In [223]:
p1 = pdPipeline([
    ("Var select", VarSelector(cat_vars)),
    ("Make Dummies", get_dummies_Pipe(10000))
])

p2 = pdPipeline([
    ("var select", VarSelector(all_other_vars))
])

dataPipe = FeatureUnion([
    ("Dummies pipeline", p1),
    ("Fake step", p2)
])


In [224]:
dataPipe.fit(trainingdata)

FeatureUnion(n_jobs=None,
             transformer_list=[('Dummies pipeline',
                                pdPipeline(memory=None,
                                           steps=[('Var select',
                                                   VarSelector(var_names=None)),
                                                  ('Make Dummies',
                                                   get_dummies_Pipe(freq_cutoff=10000))],
                                           verbose=False)),
                               ('Fake step',
                                pdPipeline(memory=None,
                                           steps=[('var select',
                                                   VarSelector(var_names=None))],
                                           verbose=False))],
             transformer_weights=None, verbose=False)

In [225]:
l =dataPipe.get_feature_names()
len(l)

65

In [226]:
x_train = pd.DataFrame(data=dataPipe.transform(trainingdata),
                       columns=dataPipe.get_feature_names())

In [227]:
x_test = pd.DataFrame(data=dataPipe.transform(testdata),
                       columns=dataPipe.get_feature_names())

In [228]:
'''All the data is now ready to be classified'''
from sklearn.model_selection import train_test_split
x_train2, x_validation = train_test_split(x_train, test_size= 0.2)

In [229]:
x_train2.head()

,Dummies pipeline__Company_Bank of America,Dummies pipeline__Company_Wells Fargo & Company,Dummies pipeline__Company_JPMorgan Chase & Co.,Dummies pipeline__Company_Equifax,Dummies pipeline__Company_Experian,"Dummies pipeline__Company_TransUnion Intermediate Holdings, Inc.",Dummies pipeline__Company_Citibank,Dummies pipeline__Company_Ocwen,Dummies pipeline__Company_Capital One,Dummies pipeline__Company_Nationstar Mortgage,...,Dummies pipeline__Sub-product_Non-federal student loan,Dummies pipeline__Sub-product_Medical,Dummies pipeline__Sub-product_Vehicle loan,Dummies pipeline__Submitted via_Web,Dummies pipeline__Submitted via_Referral,Dummies pipeline__Submitted via_Phone,Dummies pipeline__Submitted via_Postal mail,Fake step__Consumer disputed?,Fake step__Timely response?,Fake step__delay
264358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,No,1,3
237764,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,No,1,0
21959,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,No,1,0
292854,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,No,1,0
431242,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,No,1,0


In [249]:
def pavanMakeDummies(df, column, cutoff):
    k = pd.value_counts(df[column])
    df2 = df.copy()
    cats = k.index[k>cutoff][:-1]
    temp_dict = {}
    for cat in cats:
        new_name = column+'_'+cat
        df2[new_name] = (df[column]==cat).astype(int)
    del df2[column]
    
    return df2
temp = pavanMakeDummies(fulldata, 'Company', 15000) #Theres no difference wether it's 1000 or 20000.
temp.head()
    #So this works properly


,Company response to consumer,Consumer disputed?,Issue,Product,State,Sub-product,Submitted via,Timely response?,data,delay,Company_Bank of America,Company_Wells Fargo & Company,Company_JPMorgan Chase & Co.,Company_Equifax,Company_Experian,"Company_TransUnion Intermediate Holdings, Inc.",Company_Citibank,Company_Ocwen
0,Closed with explanation,No,Billing statement,Credit card,MI,missing,Web,1,train,1,0,1,0,0,0,0,0,0
1,Closed,No,"Making/receiving payments, sending money",Bank account or service,PA,(CD) Certificate of deposit,Referral,1,train,6,0,0,0,0,0,0,0,0
2,Closed with non-monetary relief,No,Incorrect information on credit report,Credit reporting,CA,missing,Referral,1,train,21,0,0,0,1,0,0,0,0
3,Closed with monetary relief,No,Billing statement,Credit card,GA,missing,Web,1,train,0,0,0,0,0,0,0,0,0
4,Closed with explanation,No,Transaction issue,Credit card,MA,missing,Web,1,train,8,1,0,0,0,0,0,0,0


In [252]:
k = pd.value_counts(fulldata['Company'])
k2 = k.index[k>10000]
print(k2,len(k2))
print(sum(k[0:8]))


Index(['Bank of America', 'Wells Fargo & Company', 'JPMorgan Chase & Co.',
       'Equifax', 'Experian', 'TransUnion Intermediate Holdings, Inc.',
       'Citibank', 'Ocwen', 'Capital One', 'Nationstar Mortgage',
       'Synchrony Financial', 'U.S. Bancorp'],
      dtype='object') 12
285972


In [254]:
backup = fulldata.copy() '''Creatinga backup before I use my custom dummy function'''

In [255]:
for col in cat_vars:
    fulldata = pavanMakeDummies(fulldata, col,20000)
fulldata.shape

(598027, 41)

In [305]:
fulldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598027 entries, 0 to 119605
Data columns (total 41 columns):
Consumer disputed?                                              598027 non-null int64
Timely response?                                                598027 non-null int64
data                                                            598027 non-null object
delay                                                           598027 non-null int64
Company_Bank of America                                         598027 non-null int64
Company_Wells Fargo & Company                                   598027 non-null int64
Company_JPMorgan Chase & Co.                                    598027 non-null int64
Company_Equifax                                                 598027 non-null int64
Company_Experian                                                598027 non-null int64
Company_TransUnion Intermediate Holdings, Inc.                  598027 non-null int64
Company_Citibank            

In [278]:
fulldata['Consumer disputed?'] = (fulldata['Consumer disputed?']=="Yes").astype(int)

In [285]:
pd.value_counts(fulldata['data'])
xtrain = fulldata.loc[fulldata['data']=="train"]
xtest = fulldata.loc[fulldata['data']=="test"]

In [315]:
del xtrain['data']
del xtest['data']

KeyError: 'data'

In [316]:
xtrain2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382736 entries, 420240 to 268909
Data columns (total 40 columns):
Consumer disputed?                                              382736 non-null int64
Timely response?                                                382736 non-null int64
delay                                                           382736 non-null int64
Company_Bank of America                                         382736 non-null int64
Company_Wells Fargo & Company                                   382736 non-null int64
Company_JPMorgan Chase & Co.                                    382736 non-null int64
Company_Equifax                                                 382736 non-null int64
Company_Experian                                                382736 non-null int64
Company_TransUnion Intermediate Holdings, Inc.                  382736 non-null int64
Company_Citibank                                                382736 non-null int64
Company response to cons

In [294]:
'''Now split into training and validation set'''
from sklearn.model_selection import train_test_split
xtrain2, xvalid = train_test_split(xtrain, test_size = 0.2)
print(xtrain2.shape[0]+xvalid.shape[0]==xtrain.shape[0])

True


In [299]:
pd.value_counts(ytrain2)

0    301882
1     80854
Name: Consumer disputed?, dtype: int64

In [296]:
ytrain2 = xtrain2['Consumer disputed?']
yvalid = xvalid['Consumer disputed?']

In [318]:
xtrain2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382736 entries, 420240 to 268909
Data columns (total 40 columns):
Consumer disputed?                                              382736 non-null int64
Timely response?                                                382736 non-null int64
delay                                                           382736 non-null int64
Company_Bank of America                                         382736 non-null int64
Company_Wells Fargo & Company                                   382736 non-null int64
Company_JPMorgan Chase & Co.                                    382736 non-null int64
Company_Equifax                                                 382736 non-null int64
Company_Experian                                                382736 non-null int64
Company_TransUnion Intermediate Holdings, Inc.                  382736 non-null int64
Company_Citibank                                                382736 non-null int64
Company response to cons

In [323]:
rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                            max_depth=10, max_features=20, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
                            oob_score=False, random_state=None, verbose=0,
                            warm_start=False)
rf.fit(xtrain2, ytrain2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=20, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [324]:
y_pred1 = rf.predict(xvalid)

In [327]:
len(y_pred1)

95685

In [342]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import fbeta_score
score = roc_auc_score(y_true=yvalid, y_score=y_pred1)
score2 = fbeta_score(y_true=yvalid, y_pred=y_pred1,beta=1)
score2
'''All well, now to predict for the test data'''

1.0

In [349]:
pd.value_counts(y_pred1)

0    75108
1    20577
dtype: int64

In [350]:
training_target = xtrain['Consumer disputed?']
del xtrain['Consumer disputed?']

In [355]:
rf.fit(xtrain, training_target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=20, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [360]:
del xtest['Consumer disputed?']

In [361]:
test_pred = rf.predict(xtest)
pd.value_counts(test_pred)

0    119605
1         1
dtype: int64

In [362]:
pd.value_counts(training_target)

0    376990
1    101431
Name: Consumer disputed?, dtype: int64

In [364]:
submission=pd.DataFrame(list(zip(testd['Complaint ID'],list(test_pred))),
                       columns=['Complaint ID','Consumer disputed?'])

In [420]:
submission['Consumer disputed?'] = np.where(submission['Consumer disputed?']==1,"Yes","No")

In [367]:
submission.to_csv("Submission1_11_feb.csv", index=False)

In [375]:
'''Trying out XGBoost'''
import xgboost as xgb
xgbc_pav = xgb.XGBClassifier()
xgbc_pav.fit(xtrain,training_target)

[08:56:39] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [378]:
xgb_pred = xgbc_pav.predict(xtest)
pd.value_counts(xgb_pred)

0    119567
1        39
dtype: int64

In [379]:
submission=pd.DataFrame(list(zip(testd['Complaint ID'],list(xgb_pred))),
                       columns=['Complaint ID','Consumer disputed?'])

In [381]:
submission.to_csv("submission2.csv",index=False)

In [404]:
'''Now to use xgb with a balanced class data'''
class0_data = xtrain.loc[xtrain['Consumer disputed?']==0]
class1_data = xtrain.loc[xtrain['Consumer disputed?']==1]

In [406]:
class0_data = class0_data.sample(101431)
class0_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101431 entries, 466047 to 200149
Data columns (total 40 columns):
Timely response?                                                101431 non-null int64
delay                                                           101431 non-null int64
Company_Bank of America                                         101431 non-null int64
Company_Wells Fargo & Company                                   101431 non-null int64
Company_JPMorgan Chase & Co.                                    101431 non-null int64
Company_Equifax                                                 101431 non-null int64
Company_Experian                                                101431 non-null int64
Company_TransUnion Intermediate Holdings, Inc.                  101431 non-null int64
Company_Citibank                                                101431 non-null int64
Company response to consumer_Closed with explanation            101431 non-null int64
Company response to cons

In [410]:
xtrain_balanced = pd.concat([class0_data,class1_data],axis=0)

In [413]:
pd.value_counts(xtrain_balanced['Consumer disputed?'])
training_target_balanced = xtrain_balanced['Consumer disputed?']
del xtrain_balanced['Consumer disputed?']

In [414]:
classifier = xgb.XGBClassifier()
classifier.fit(xtrain_balanced, training_target_balanced)

/home/pavan117/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:19:34] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [416]:
xtest.shape

(119606, 39)

In [417]:
test_pred_balanced = classifier.predict(xtest)
pd.value_counts(test_pred_balanced)

1    63680
0    55926
dtype: int64

In [418]:
submission=pd.DataFrame(list(zip(testd['Complaint ID'],list(test_pred_balanced))),
                       columns=['Complaint ID','Consumer disputed?'])
submission.to_csv("Balanced_Pred.csv",index=False)

In [422]:
pd.value_counts(submission['Consumer disputed?'])

Yes    63680
No     55926
Name: Consumer disputed?, dtype: int64